In [32]:
import argparse
from copy import deepcopy
import logging
import random
from collections import defaultdict
from os.path import join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, f1_score, precision_recall_curve
from sklearn.model_selection import train_test_split
import joblib
import imodels
import inspect
import os.path
import imodelsx.cache_save_utils
import sys
import torch

#path_to_repo = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

#os.chdir(path_to_repo)
#os.chdir('/home/mattyshen/interpretableDistillation')
sys.path.append('..')

import idistill.model
import idistill.data

def distill_model(student, X_train_teacher, y_train_teacher, r, feature_names = None):
    """Distill the teacher model using the student model"""
    
    fit_parameters = inspect.signature(student.fit).parameters.keys()
    if "feature_names" in fit_parameters and feature_names is not None:
        student.fit(X_train_teacher, y_train_teacher, feature_names=feature_names)
    else:
        student.fit(X_train_teacher, y_train_teacher)

    return r, student

def evaluate_student(student, X_train, X_test, y_train, y_test, metric, task, r):
    """Evaluate student performance on each split"""
    
    metrics = {
            "accuracy": accuracy_score,
            "mse": mean_squared_error,
            "r2": r2_score,
            "f1": f1_score,
        
        }
    
    metric_fn = metrics[metric]
    
    for split_name, (X_, y_) in zip(
        ["train", "test"], [(X_train, y_train), (X_test, y_test)]
    ):
        y_pred_ = process_student_eval(student.predict(X_))
        r[f"student_{task}_{split_name}_{metric}"] = metric_fn(y_, y_pred_)

    return r

def evaluate_teacher(y_train_teacher, y_test_teacher, y_train, y_test, metric, task, r):
    metrics = {
            "accuracy": accuracy_score,
            "mse": mean_squared_error,
            "r2": r2_score,
            "f1": f1_score,
        
        }
    
    metric_fn = metrics[metric]
    
    for split_name, (y_teacher_, y_) in zip(
        ["train", "test"], [(y_train_teacher, y_train), (y_test_teacher, y_test)]
    ):
        r[f"teacher_{task}_{split_name}_{metric}"] = metric_fn(y_teacher_, y_)
    
    return r

def predict_teacher(teacher, X, gpu=0):
    ### TODO: handle teacher prediction outputs (X is intended to be concept design matrix, output is intended to be logits)###

    y_pred_torch = teacher.sec_model(torch.tensor(X.values, dtype=torch.float32).to(f'cuda:{gpu}'))
    y_pred = pd.DataFrame(y_pred_torch.detach().cpu().numpy())
        
    return y_pred

def load_teacher_model(teacher_path, gpu=0):
    ### TODO: load in teacher model using teacher_path ###
    
    sys.path.append('/home/mattyshen/ConceptBottleneck')
    try:
        teacher = torch.load(teacher_path, weights_only=False)
    except:
        teacher = torch.load(join(join('home/mattyshen/DistillationEdit/', "models"), teacher_path), weights_only=False)
    teacher.to(f'cuda:{gpu}')
    teacher.eval()
    sys.path.append('/home/mattyshen/DistillationEdit')
    
    return teacher

def generate_tabular_distillation_data(teacher, train_path, test_path, gpu=0):
    ### TODO: generate teacher train and test data using model, train_path, and test_path ###
    
    sys.path.append('/home/mattyshen/ConceptBottleneck/CUB')
    from dataset import load_data
    from config import BASE_DIR
    
    def get_cub_data(teacher, path, data = 'train', override_train = True, batch_size = 32):
        with torch.no_grad():
            if data == 'test':
                test_dir = path
                #print(test_dir)
                # loader = load_data([test_dir], True, False, batch_size, image_dir='images',
                #                    n_class_attr=2, override_train=override_train)
                loader = load_data([test_dir], True, False, batch_size, image_dir='images',
                                   n_class_attr=2)
            else:
                train_dir = path
                val_dir = '/home/mattyshen/ConceptBottleneck/CUB_processed/class_attr_data_10/val.pkl'
                #print(train_dir, val_dir)
                # loader = load_data([train_dir, val_dir], True, False, batch_size, image_dir='images',
                #                    n_class_attr=2, override_train=override_train)
                loader = load_data([train_dir, val_dir], True, False, batch_size, image_dir='images',
                                    n_class_attr=2)
                
            torch.manual_seed(0)
            
            attrs_true = []
            attrs_hat = []
            labels_true = []
            labels_hat = []
            for data_idx, data in enumerate(loader):
                inputs, labels, attr_labels = data
                attr_labels = torch.stack(attr_labels).t()

                inputs_var = torch.autograd.Variable(inputs).to(f'cuda:{gpu}')
                labels_var = torch.autograd.Variable(labels).to(f'cuda:{gpu}')
                outputs = teacher(inputs_var)
                class_outputs = outputs[0]

                attr_outputs = [torch.nn.Sigmoid()(o) for o in outputs[1:]] #outputs[1:]
                #attr_outputs_sigmoid = attr_outputs

                attrs_hat.append(torch.stack(attr_outputs).squeeze(2).detach().cpu().numpy())
                attrs_true.append(attr_labels.T)
                labels_hat.append(class_outputs.detach().cpu().numpy())
                labels_true.append(labels)

            X_hat = pd.DataFrame(np.concatenate(attrs_hat, axis=1).T, columns = [f'c{i}' for i in range(1, 113)])
            X = pd.DataFrame(np.concatenate(attrs_true, axis = 1).T, columns = [f'c{i}' for i in range(1, 113)])

            y = pd.Series(np.concatenate([l.numpy().reshape(-1, ) for l in labels_true]))
            y_hat = pd.DataFrame(np.concatenate(labels_hat, axis = 0))

            del attrs_hat
            del labels
            del labels_hat
            del loader
            del data
            del inputs
            del outputs
            del class_outputs
            del attr_outputs
            del inputs_var
            del labels_var
            torch.cuda.empty_cache()

            return X_hat, X, y_hat, y

    X_train_teacher, X_train, y_train_teacher, y_train = get_cub_data(teacher, train_path)
    X_test_teacher, X_test, y_test_teacher, y_test = get_cub_data(teacher, test_path, data = 'test')
    
    sys.path.append('/home/mattyshen/DistillationEdit')
    
    return X_train_teacher, X_test_teacher, X_train, X_test, y_train_teacher, y_test_teacher, y_train, y_test
    
def process_distillation_data(X_train_teacher, X_test_teacher, X_train, X_test, y_train_teacher, y_test_teacher):
    ### TODO: process (i.e. binarize, F1-max binarize) data for distillation ###
    
#     thresh = 0
    
#     return (X_train_teacher > thresh).astype(int), (X_test_teacher > thresh).astype(int), y_train_teacher, y_test_teacher

    best_t = np.argmax([np.mean((X_train_teacher.values > t).astype(int) == X_train.values) for t in np.arange(0, 1, 0.01)])
    thresh = np.arange(0, 1, 0.01)[best_t]
    
    return (X_train_teacher > thresh).astype(int), (X_test_teacher > thresh).astype(int), y_train_teacher, y_test_teacher

def process_student_eval(y_student):
    ### TODO: handle student prediction outputs to match metrics ###
    
    y_pred = np.argmax(y_student, axis = 1)

    return y_pred

def process_teacher_eval(y_teacher):
    ### TODO: process teacher model predictions for evaluations (sometimes we distill a teacher model using a regressor, but want to evaluate class prediction accuracy) ###
    
    y_teacher_eval = y_teacher.idxmax(axis = 1).astype(int).values
    
    return y_teacher_eval

def extract_interactions(student):

    interactions = []

    def traverse_tree(node, current_features, current_depth):

        if node.left is None and node.right is None:
            tree_interactions.append((current_features, np.var(np.abs(node.value))))
            return
        if node.left is not None:
            current_features_l = current_features.copy()
            current_features_l.append('c' + str(node.feature+1))
            traverse_tree(node.left, current_features_l.copy(), current_depth=current_depth+1)
        if node.right is not None:
            current_features_r = current_features.copy()
            current_features_r.append('!c' + str(node.feature+1))
            traverse_tree(node.right, current_features_r.copy(), current_depth=current_depth+1)
            
    try:
        trees = student.trees_
    except:
        trees = student.figs.trees_

    for tree in trees:
        tree_interactions = []
        traverse_tree(tree, [], current_depth=0)
        interactions.append(tree_interactions)
        
    return interactions

def get_argmax_max(vals, index):
    
    maxes = np.partition(vals, -2, axis=1)[:, -index]
    argmaxes = np.argsort(vals, axis=1)[:, -index]
    return maxes, argmaxes

def extract_adaptive_intervention(student, X, interactions, number_of_top_paths, tol = 0.0001):
    
    test_pred_intervention = student.predict(X, by_tree = True)

    concepts_to_edit = [[] for _ in range(X.shape[0])]
    variances = np.var(np.abs(test_pred_intervention), axis = 1)

    for idx in range(number_of_top_paths):
        maxes, argmaxes = get_argmax_max(variances, idx+1)
        for i, (tree_idx, var) in enumerate(zip(argmaxes, maxes)):
            for paths in interactions[tree_idx]:
                if abs(paths[1] - var) < tol:
                    concept_indexes = [int(p[1:])-1 if p[0] != '!' else int(p[2:])-1 for p in paths[0]]
                    concepts_to_edit[i].append(concept_indexes)
                    
    concepts_to_edit = [sum(element, []) for element in concepts_to_edit]
    concepts_to_edit = [list(set(c)) for c in concepts_to_edit]
    
    return concepts_to_edit

In [13]:
class ARGS:
    def __init__(self, a_dict):
        for k in a_dict.keys():
            exec(f'self.{k} = a_dict["{k}"]')
        

In [36]:
args = {}
#args['save_dir'] = join(path_to_repo, "results")  # The default value
args['teacher_path'] = '/home/mattyshen/DistillationEdit/models/cub_sigmoid/outputs/best_Joint0.01_Linear_model_1.pth'  # The default value
args['train_path'] = '/home/mattyshen/ConceptBottleneck/CUB_processed/class_attr_data_10/train.pkl'  # The default value
args['test_path'] = '/home/mattyshen/ConceptBottleneck/CUB_processed/class_attr_data_10/test.pkl'  # The default value
args['task_type'] = "regression"  # The default value
args['student_name'] = "FIGSRegressor"  # The default value
args['max_rules'] = 125  # The default value
args['max_trees'] = 25  # The default value
args['max_depth'] = 4  # The default value
args['min_impurity_decrease'] = 0  # The default value
args['metric'] = "accuracy"  # The default value
args['num_interactions_intervention'] = 3  # The default value
args['n_trees_list'] = [30,40]
args['n_rules_list'] = [125, 150, 200]
args['n_depth_list'] = [4]
args['min_impurity_decrease_list'] = [0]
args['gpu'] = 0

args = ARGS(args)

In [15]:
r = defaultdict(list)

In [16]:
teacher = load_teacher_model(args.teacher_path, args.gpu)
    
X_train_t, X_test_t, X_train, X_test, y_train_t, y_test_t, y_train, y_test = generate_tabular_distillation_data(teacher, args.train_path, args.test_path, args.gpu)

X_train_d, X_test_d, y_train_d, y_test_d = process_distillation_data(X_train_t, X_test_t, X_train, X_test, y_train_t, y_test_t)

y_train_t_eval = process_teacher_eval(y_train_t)
y_test_t_eval = process_teacher_eval(y_test_t)

/home/mattyshen/anaconda3/lib/python3.9/site-packages/torch/serialization.py:1189: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/mattyshen/anaconda3/lib/python3.9/site-packages/torch/serialization.py:1189: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/mattyshen/anaconda3/lib/python3.9/site-packages/torch/serialization.py:1189: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source c

In [124]:
X_train_d, X_test_d, y_train_d, y_test_d = process_distillation_data(X_train_t, X_test_t, X_train, X_test, y_train_t, y_test_t)

y_train_t_eval = process_teacher_eval(y_train_t)
y_test_t_eval = process_teacher_eval(y_test_t)

In [125]:
from idistill.whitebox_figs import FIGSRegressor

#figs_student = idistill.model.get_model(args.task_type, args.student_name, args)

In [154]:
figs_student = FIGSRegressor(max_rules=args.max_rules, 
                                    max_trees=args.max_trees, 
                                    max_depth=args.max_depth)

In [155]:
figs_student

FIGSRegressor(max_depth=4, max_rules=125, max_trees=25)

In [156]:
r, figs_student = distill_model(figs_student, X_train_d, y_train_d, r)

In [157]:
r = evaluate_student(figs_student, X_train_d, X_test_d, y_train_t_eval, y_test_t_eval, args.metric, "distillation", r)
r = evaluate_student(figs_student, X_train_d, X_test_d, y_train, y_test, args.metric, "prediction", r)

r = evaluate_teacher(y_train_t_eval, y_test_t_eval, y_train, y_test, args.metric, "prediction", r)

In [158]:
r

defaultdict(list,
            {'student_distillation_train_accuracy': 0.9705882352941176,
             'student_distillation_test_accuracy': 0.9342423196410079,
             'student_prediction_train_accuracy': 0.9406751336898396,
             'student_prediction_test_accuracy': 0.7576803589920608,
             'teacher_prediction_train_accuracy': 0.9476938502673797,
             'teacher_prediction_test_accuracy': 0.7704521919226787,
             'student_distillation_iter_train_accuracy': 0.9627339572192514,
             'student_distillation_iter_test_accuracy': 0.9190541939937866,
             'student_prediction_iter_train_accuracy': 0.9351604278074866,
             'student_prediction_iter_test_accuracy': 0.7537107352433552,
             'teacher_prediction_iter_train_accuracy': 0.9476938502673797,
             'teacher_prediction_iter_test_accuracy': 0.7704521919226787})

In [159]:
figs_test_preds = figs_student.predict(X_test_d)

In [160]:
np.argmax(figs_test_preds, axis = 1)

array([ 70,   0,   0, ..., 199, 199, 199])

In [161]:
comp = pd.DataFrame([np.argmax(figs_test_preds, axis = 1), y_test_t.idxmax(axis=1)]).T

In [162]:
comp['eq'] = comp[0] == comp[1]

In [163]:
import matplotlib.pyplot as plt

In [164]:
mapping = {}
for k, v in comp[comp['eq'] == False][1].value_counts()[:50].reset_index().values:
    mapping[k]= v

/tmp/ipykernel_5740/43315300.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for k, v in comp[comp['eq'] == False][1].value_counts()[:50].reset_index().values:


In [165]:
training = pd.DataFrame(y_train)
training['weight']  = training[0].map(mapping)
training.fillna(0, inplace=True)

In [166]:
from sklearn.linear_model import LogisticRegression

In [167]:
lr = LogisticRegression()
lr.fit(X_test_d, comp['eq'])

LogisticRegression()

In [168]:
comp

,0,1,eq
0,70,70,True
1,0,0,True
2,0,0,True
3,0,0,True
4,0,0,True
...,...,...,...
5789,199,199,True
5790,199,199,True
5791,199,199,True
5792,199,199,True


In [169]:
lr.predict_proba(X_train_d)[:, 0]

array([0.20008731, 0.05103759, 0.02728497, ..., 0.00392148, 0.00392148,
       0.02503763])

In [170]:
training['weight']

0       9.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5979    0.0
5980    0.0
5981    0.0
5982    0.0
5983    0.0
Name: weight, Length: 5984, dtype: float64

In [171]:
new_weight = training['weight']+ 1 #((training['weight']/training['weight'].median()) + 1).to_numpy()

In [172]:
figs_student = FIGSRegressor(max_rules=args.max_rules, 
                                    max_trees=args.max_trees, 
                                    max_depth=args.max_depth)

figs_student.fit(X_train_t, y_train_t, sample_weight = lr.predict_proba(X_train_d)[:, 0])

FIGSRegressor(max_depth=4, max_rules=125, max_trees=25)

In [173]:
r = evaluate_student(figs_student, X_train_d, X_test_d, y_train_t_eval, y_test_t_eval, args.metric, "distillation_iter", r)
r = evaluate_student(figs_student, X_train_d, X_test_d, y_train, y_test, args.metric, "prediction_iter", r)

r = evaluate_teacher(y_train_t_eval, y_test_t_eval, y_train, y_test, args.metric, "prediction_iter", r)

In [174]:
r

defaultdict(list,
            {'student_distillation_train_accuracy': 0.9705882352941176,
             'student_distillation_test_accuracy': 0.9342423196410079,
             'student_prediction_train_accuracy': 0.9406751336898396,
             'student_prediction_test_accuracy': 0.7576803589920608,
             'teacher_prediction_train_accuracy': 0.9476938502673797,
             'teacher_prediction_test_accuracy': 0.7704521919226787,
             'student_distillation_iter_train_accuracy': 0.955548128342246,
             'student_distillation_iter_test_accuracy': 0.9097342078011736,
             'student_prediction_iter_train_accuracy': 0.9288101604278075,
             'student_prediction_iter_test_accuracy': 0.7454263030721436,
             'teacher_prediction_iter_train_accuracy': 0.9476938502673797,
             'teacher_prediction_iter_test_accuracy': 0.7704521919226787})

In [ ]:
figs_interactions = extract_interactions(figs_student)

X_train_d_edit = X_train_d.copy()
X_train_t_edit = X_train_t.copy()

train_q5 = np.quantile(X_train_t, 0.05, axis = 0)
train_q95 = np.quantile(X_train_t, 0.95, axis = 0)

# train_q5d = np.quantile(X_train_d, 0.05, axis = 0)
# train_q95d = np.quantile(X_train_d, 0.95, axis = 0)

cti_train = extract_adaptive_intervention(figs_student, X_train_d, figs_interactions, args.num_interactions_intervention)

for i in range(len(cti_train)):
    X_train_d_edit.iloc[i, cti_train[i]] = X_train.iloc[i, cti_train[i]] #train_q5d[cti_train[i]]*(X_train.iloc[i, cti_train[i]] == 0) + train_q95d[cti_train[i]]*(X_train.iloc[i, cti_train[i]])
    X_train_t_edit.iloc[i, cti_train[i]] = train_q5[cti_train[i]]*(X_train.iloc[i, cti_train[i]] == 0) + train_q95[cti_train[i]]*(X_train.iloc[i, cti_train[i]])

cti_test = extract_adaptive_intervention(figs_student, X_test_d, figs_interactions, args.num_interactions_intervention)

X_test_d_edit = X_test_d.copy()
X_test_t_edit = X_test_t.copy()

for i in range(len(cti_test)):
    X_test_d_edit.iloc[i, cti_test[i]] = X_test.iloc[i, cti_test[i]] #train_q5d[cti_test[i]]*(X_test.iloc[i, cti_test[i]] == 0) + train_q95d[cti_test[i]]*(X_test.iloc[i, cti_test[i]])
    X_test_t_edit.iloc[i, cti_test[i]] = train_q5[cti_test[i]]*(X_test.iloc[i, cti_test[i]] == 0) + train_q95[cti_test[i]]*(X_test.iloc[i, cti_test[i]])

y_train_t_eval_interv = process_teacher_eval(predict_teacher(teacher, X_train_t_edit))
y_test_t_eval_interv = process_teacher_eval(predict_teacher(teacher, X_test_t_edit))

r = evaluate_student(figs_student, X_train_d_edit, X_test_d_edit, y_train_t_eval_interv, y_test_t_eval_interv, args.metric, "distillation_adap_interv", r)
r = evaluate_student(figs_student, X_train_d_edit, X_test_d_edit, y_train, y_test, args.metric, "prediction_adap_interv", r)

r = evaluate_teacher(y_train_t_eval_interv, y_test_t_eval_interv, y_train, y_test, args.metric, "prediction_adap_interv", r)

In [ ]:
args.num_interactions_intervention * args.max_depth

In [ ]:
plt.hist([len(i) for i in cti_train])

In [ ]:
plt.hist([len(i) for i in cti_test])

In [ ]:
cti_r_train = [np.random.choice(np.arange(0, 112), size=len(c), replace=False) for c in cti_train]
cti_r_test = [np.random.choice(np.arange(0, 112), size=len(c), replace=False) for c in cti_test]

In [ ]:
plt.hist([len(i) for i in cti_r_train])

In [ ]:
plt.hist([len(i) for i in cti_r_test])

In [ ]:
X_train_t

In [ ]:
X_train_d_r_edit = X_train_d.copy()
X_train_t_r_edit = X_train_t.copy()

for i in range(len(cti_r_train)):
    X_train_d_r_edit.iloc[i, cti_r_train[i]] = X_train.iloc[i, cti_r_train[i]] #train_q5d[cti_r_train[i]]*(X_train.iloc[i, cti_r_train[i]] == 0) + train_q95d[cti_r_train[i]]*(X_train.iloc[i, cti_r_train[i]])
    X_train_t_r_edit.iloc[i, cti_r_train[i]] = train_q5[cti_r_train[i]]*(X_train.iloc[i, cti_r_train[i]] == 0) + train_q95[cti_r_train[i]]*(X_train.iloc[i, cti_r_train[i]])

X_test_d_r_edit = X_test_d.copy()
X_test_t_r_edit = X_test_t.copy()

for i in range(len(cti_r_test)):
    X_test_d_r_edit.iloc[i, cti_r_test[i]] = X_test.iloc[i, cti_r_test[i]] #train_q5d[cti_r_test[i]]*(X_test.iloc[i, cti_r_test[i]] == 0) + train_q95d[cti_r_test[i]]*(X_test.iloc[i, cti_r_test[i]])
    X_test_t_r_edit.iloc[i, cti_r_test[i]] = train_q5[cti_r_test[i]]*(X_test.iloc[i, cti_r_test[i]] == 0) + train_q95[cti_r_test[i]]*(X_test.iloc[i, cti_r_test[i]])

y_train_t_eval_r_interv = process_teacher_eval(predict_teacher(teacher, X_train_t_r_edit))
y_test_t_eval_r_interv = process_teacher_eval(predict_teacher(teacher, X_test_t_r_edit))

r = evaluate_student(figs_student, X_train_d_r_edit, X_test_d_r_edit, y_train_t_eval_r_interv, y_test_t_eval_r_interv, args.metric, "distillation_rand_interv", r)
r = evaluate_student(figs_student, X_train_d_r_edit, X_test_d_r_edit, y_train, y_test, args.metric, "prediction_rand_interv", r)

r = evaluate_teacher(y_train_t_eval_r_interv, y_test_t_eval_r_interv, y_train, y_test, args.metric, "prediction_rand_interv", r)

In [ ]:
r

In [ ]:
train_l_edit = np.einsum('nc, yc -> nyc', X_train_t.values, teacher.sec_model.linear.weight.cpu().detach().numpy())
test_l_edit = np.einsum('nc, yc -> nyc', X_test_t.values, teacher.sec_model.linear.weight.cpu().detach().numpy())

cti_l_train_arr = np.argsort(np.max(train_l_edit, axis = 1), axis = 1)[:, -9:]
cti_l_train = [row for row in cti_l_train_arr]

cti_l_test_arr = np.argsort(np.max(test_l_edit, axis = 1), axis = 1)[:, -9:]
cti_l_test = [row for row in cti_l_test_arr]

In [ ]:
X_train_d_l_edit = X_train_d.copy()
X_train_t_l_edit = X_train_t.copy()

for i in range(len(cti_l_train)):
    X_train_d_l_edit.iloc[i, cti_l_train[i]] = X_train.iloc[i, cti_l_train[i]] #train_q5d[cti_l_train[i]]*(X_train.iloc[i, cti_l_train[i]] == 0) + train_q95d[cti_l_train[i]]*(X_train.iloc[i, cti_l_train[i]])
    X_train_t_l_edit.iloc[i, cti_l_train[i]] = train_q5[cti_l_train[i]]*(X_train.iloc[i, cti_l_train[i]] == 0) + train_q95[cti_l_train[i]]*(X_train.iloc[i, cti_l_train[i]])

X_test_d_l_edit = X_test_d.copy()
X_test_t_l_edit = X_test_t.copy()

for i in range(len(cti_l_test)):
    X_test_d_l_edit.iloc[i, cti_l_test[i]] = X_test.iloc[i, cti_l_test[i]] #train_q5d[cti_l_test[i]]*(X_test.iloc[i, cti_l_test[i]] == 0) + train_q95d[cti_l_test[i]]*(X_test.iloc[i, cti_l_test[i]])
    X_test_t_l_edit.iloc[i, cti_l_test[i]] = train_q5[cti_l_test[i]]*(X_test.iloc[i, cti_l_test[i]] == 0) + train_q95[cti_l_test[i]]*(X_test.iloc[i, cti_l_test[i]])

y_train_t_eval_r_interv = process_teacher_eval(predict_teacher(teacher, X_train_t_l_edit))
y_test_t_eval_r_interv = process_teacher_eval(predict_teacher(teacher, X_test_t_l_edit))

r = evaluate_student(figs_student, X_train_d_l_edit, X_test_d_l_edit, y_train_t_eval_r_interv, y_test_t_eval_r_interv, args.metric, "distillation_lin_interv", r)
r = evaluate_student(figs_student, X_train_d_l_edit, X_test_d_l_edit, y_train, y_test, args.metric, "prediction_lin_interv", r)

r = evaluate_teacher(y_train_t_eval_r_interv, y_test_t_eval_r_interv, y_train, y_test, args.metric, "prediction_lin_interv", r)

In [ ]:
#thresh = 0.45
r

In [ ]:
#thresh = median of training values
r

In [ ]:
#thresh = 0
r

In [ ]:
len(figs_student.trees_)